This code implements the DBME method proposed and analyzed in the paper ``DEEP BACKWARD AND GALERKIN METHODS FOR LEARNING FINITE
STATE MASTER EQUATIONS'' by Asaf Cohen, Mathieu Laurière and Ethan Zell.

The example solved here corresponds to Example 7.1 in the paper.

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import math, pandas, sklearn, keras, random, copy, time
import matplotlib.pyplot as plt
tf.random.set_seed(703)
np.random.seed(703)

In [ ]:
# Global, static parameters
d=2

In [ ]:
class DBDatasetGenerator:
  '''
  This class generates the dataset for the DGM.
  '''
  def __init__(self, d=2, horizon=.5):
    self.d = d
    self.horizon = horizon

  def uniformly_random_measure(self):
    '''
    Uses exponential random variables to generate a uniformly random probability vector.
    '''
    pre_normalized = np.random.exponential(1, size = (self.d,))
    return pre_normalized / sum(pre_normalized)

  def uniformly_random_measure_vec(self, samples):
    '''
    Vectorizes the prior function.
    '''
    data = np.zeros((samples, self.d))
    for k in range(samples):
      data[k,:] = self.uniformly_random_measure()
    return data

  def generate_dataset(self, samples=1000):
    '''
    Creates the dataset which, for the DBME, needs only x and eta data.
    '''
    x_data = np.random.choice([float(m) for m in range(self.d)],size=samples)
    eta_data = self.uniformly_random_measure_vec(samples = samples)
    self.x_data = x_data
    self.eta_data = eta_data
    return x_data, eta_data

  def oversample_eta_bijection(self, x):
    return (2.* x) - 0.5

  def oversampling(self, oversample_eta = True, oversample_T = True):
    '''
    Apply this function after generate_dataset to modify the domain of the sampled eta and T.
    Sampling outside the domain may improve performance along the domain's boundary.
    '''
    if oversample_eta:
      self.eta_data = self.oversample_eta_bijection(self.eta_data)
    return self.x_data, self.eta_data

  def data_to_tensors(self):
    self.x_data = tf.convert_to_tensor(self.x_data, dtype = 'float32')
    self.eta_data = tf.convert_to_tensor(self.eta_data, dtype = 'float32')
    return self.x_data, self.eta_data

In [ ]:
class DBMFGModel(tf.keras.Model):
  '''
  This class defines the neural network model.
  '''
  def __init__(self, architecture):
    super(DBMFGModel, self).__init__()
    self.architecture = architecture # you can give a list specifying the number of nodes in each dense layer
    self.layer_list = []

    for i,number_of_nodes in enumerate(architecture):
      if i == 0:
        self.layer_list.append(tf.keras.layers.Dense(units=number_of_nodes, activation='sigmoid',
                                                              kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.),
                                                              bias_initializer='zeros'))
      else:
        self.layer_list.append(tf.keras.layers.Dense(units=number_of_nodes, activation='sigmoid',
                                                              kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.),
                                                              bias_initializer='zeros'))
    self.layer_list.append(tf.keras.layers.Dense(units=1, activation = 'elu'))

  def call(self, x, eta):
    x = tf.expand_dims(x, axis = -1)
    input = tf.concat([x, eta], 1)
    result = input
    for layer in self.layer_list:
      result = layer(result)
    return result

Recall from the paper that we are interested in approximately solving the master equation:

$$
\partial_t U(t,x,\eta) = H(x,\Delta_x U(t,\cdot,\eta))+ F(x,\eta) + \sum_{y,z\in [d]} D^\eta_{yz} U(t,x,\eta) \gamma^*_z(y,\Delta_y U(t,\cdot,\eta)) \eta_y,
$$

where in this example:

$$F(x,\eta) = \eta_x,$$

$$H(x,p) := \min_{a} \Big\{\frac{1}{2}|a|^2 + a\cdot p\Big\},$$

and where $\gamma^*$ is the associated minimal argument that minimizes the Hamiltonian $H$. Recall that $\Delta_x b:= (b_y - b_x)_{y\in [d]}$ is a finite difference vector and $D^\eta_{yz}$ denotes the directional derivative in the $z$ minus $y$ direction (in terms of the standard basis).

In the Loss class below, $F$ is referred to as the mean_field_cost and $H$ is the Hamiltonian.

In [ ]:
class Loss():
  '''
  This class defines the loss, which involves the MFG model.
  '''
  def __init__(self, model_mesh, step, partition_step, future_is_terminal = False, a = 2., a_l = 1., a_u = 3., b = 4.):
    self.model = model_mesh[step]

    if future_is_terminal:
      self.model_at_future_step = None
    else:
      self.model_at_future_step = model_mesh[step + 1]

    self.a = a
    self.a_l = a_l
    self.a_u = a_u
    self.b = b
    self.partition_step = partition_step
    self.future_is_terminal = future_is_terminal
    return

  def a_star(self, numerator):
    '''
    The computed value of $\gamma^*$, the minimal argument of the Hamiltonian. The paper derives this formula explicitly.
    '''
    return (numerator / ((self.a_u - self.a_l) * self.b) ) + self.a

  def Hamiltonian(self, output, complement_output):
    '''
    The Hamiltonian, denoted H in the paper.
    '''
    a_star_x = self.a_star(output - complement_output)
    pre_running = a_star_x - (self.a * np.ones(a_star_x.shape))
    running_cost = self.b * tf.math.square(pre_running)
    change_of_state = tf.multiply(a_star_x, (complement_output - output))
    return running_cost + change_of_state

  def mean_field_cost(self, x, eta):
    '''
    The common cost, denoted F in the paper.
    '''
    mf = np.zeros(x.shape)
    for i,entry in enumerate(x):
      mf[i] = eta[i, int(entry)]
    mfc = tf.convert_to_tensor(mf, dtype='float32')
    return mfc

  def criterion(self, x, eta):
    '''
    The loss function for the DBME, which considers the partition step and the neural network at the immediately future time step.
    '''
    output = tf.squeeze(self.model(x, eta))
    if self.future_is_terminal:
      future_output = tf.zeros(output.shape)
    else:
      future_output = tf.squeeze(self.model_at_future_step(x, eta))
    complement_output = tf.squeeze(self.model(1.-x, eta))
    hamiltonian = self.Hamiltonian(output, complement_output)
    mean_field_cost = self.mean_field_cost(x, eta)
    loss_sum = future_output - output + (self.partition_step * (hamiltonian + mean_field_cost))
    squared_loss = tf.math.square(loss_sum)

    return squared_loss

  def total_criterion(self, x, eta):
    '''
    The DBME uses a max norm in its loss.
    '''
    unreduced_loss = self.criterion(x, eta)
    loss = tf.math.reduce_max(unreduced_loss)
    return loss

In [ ]:
class Train():
  def __init__(self, model_architecture, dataset_generator, partition_step,
               oversampling = True, return_losses = False, verbose = False, visual_output = False):
    self.model_arch = model_architecture
    self.dsg = dataset_generator
    self.return_losses = return_losses
    self.losses = []
    self.verbose = verbose
    self.visual_output = visual_output
    self.partition_step = partition_step
    self.total_steps = int(self.dsg.horizon / self.partition_step) + 1
    print(f'Total number of steps will be: {self.total_steps}')
    self.nn_mesh = np.zeros(self.total_steps, dtype=object)

  def initialize_nn_mesh(self):
    # input_shape = (d+1,)
    # d dimensions for the measure and the last dimension is for the jump process
    for i in range(self.nn_mesh.shape[0]):
      self.nn_mesh[i] = DBMFGModel(architecture = self.model_arch)
    return self.nn_mesh

  def loss_gradient(self, step):
    if step >= self.total_steps - 1:
      future = True
    else:
      future = False
    loss_fn = Loss(model_mesh = self.nn_mesh, step = step, partition_step = self.partition_step,
                   future_is_terminal = future)
    with tf.GradientTape(persistent=True) as loss_tape:
      loss = loss_fn.total_criterion(self.x, self.eta)
    return loss, loss_tape.gradient(loss, self.nn_mesh[step].trainable_variables)

  def step(self, step, optimizer):

    '''
    A single step in the training regime of a particular neural network in the mesh.
    '''

    loss, loss_grad = self.loss_gradient(step = step)

    if self.verbose:
      self.avg_losses.append(loss.numpy())
    if self.return_losses:
      self.losses.append(loss)

    optimizer.apply_gradients(zip(loss_grad, self.nn_mesh[step].trainable_variables))
    return self.nn_mesh[step]

  def train(self, epochs, steps_per_epoch, learning_rate = 1e-4):

    '''
    The main training function to train each neural network in the mesh, one by one, from the terminal time until time zero.
    '''

    self.initialize_nn_mesh()
    print('Training network mesh.')

    for n in tqdm(range(self.total_steps-1, -1, -1)): # start from the terminal time and work backward

      if self.verbose:
        print('')
        print(f'----------training network {n}----------')
      if n < self.total_steps - 1:
        e = epochs
        try:
          self.nn_mesh[n].load_weights(f'weights_{n+1}') # we expect that the previous learned weights would be close to the weights in the next step, by continuity of the master equation solution
        except:
          pass
      else:
        e = epochs*3

      opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)

      for m in tqdm(range(e)):

        self.avg_losses = []
        self.dsg.generate_dataset()
        self.dsg.oversampling()
        self.x, self.eta = self.dsg.data_to_tensors()

        for s in range(steps_per_epoch):
          self.nn_mesh[n] = self.step(step = n, optimizer = opt)

        if self.verbose:
          print(f'Avg loss for epoch {m} was: {np.mean(self.avg_losses)}')

      self.nn_mesh[n].save_weights(f'weights_{n}')

    if self.return_losses:
      return self.nn_mesh, self.losses
    return self.nn_mesh

In [ ]:
trainer = Train(model_architecture = [d+1,20,20], dataset_generator = DBDatasetGenerator(), partition_step = 0.1, oversampling = True, verbose = True)
mesh = trainer.train(epochs = 5, steps_per_epoch = 2, learning_rate = 1e-4)
# For best results, increase epochs to > 150 and steps_per_epoch > 20.

Total number of steps will be: 6
Training network mesh.


  0%|          | 0/6 [00:00<?, ?it/s]


----------training network 5----------



  0%|          | 0/15 [00:00<?, ?it/s]WARNING:tensorflow:5 out of the last 5 calls to <function _BaseOptimizer._update_step_xla at 0x7c1a3a6724d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.

  7%|▋         | 1/15 [00:13<03:03, 13.13s/it]

Avg loss for epoch 0 was: 0.4934001863002777


...

Avg loss for epoch 2 was: 0.023756545037031174



 80%|████████  | 4/5 [00:09<00:02,  2.41s/it]

Avg loss for epoch 3 was: 0.022652970626950264



100%|██████████| 6/6 [01:39<00:00, 16.64s/it]

Avg loss for epoch 4 was: 0.021581776440143585


Below are the plotting functions used for the DBME in the paper.

In [ ]:
from matplotlib.pyplot import cm
import seaborn as sns
import imageio
import copy
import matplotlib as mpl
import plotly.graph_objects as go

There are two visualization classes. The first allows comparison of one output of the DGME method with one output of the DBME method. To get the plot comparing multiple partition sizes for the DBME output to the DGME output, we use the class VizSeveral in the latter cell. As noted in the paper, in practice, the models were computed on Michigan's Great Lakes advanced computing cluster, downloaded, and then loaded into these visualization classes.

In the first class, VizBoth, the second input is a list of neural networks corresponding to the mesh output of a single DBME run. However, in VizSeveral, the input is an array of neural networks, corresponding to several meshes, and several DBME runs with different partition steps.

In [ ]:
class VizBoth:
  def __init__(self, dgm_model, dbmfg_model_list, num_measure_points =1_000):
    self.dgm_model = dgm_model
    self.dbmfg_model_list = dbmfg_model_list
    self.num_points = num_measure_points

  def single_dgm_graph_population(self, t=0., x=0.):
    two_simplex = np.linspace(start = 0, stop = 1, num = self.num_points, endpoint = True)
    data_for_graph = np.zeros((self.num_points, d+2))
    eta1 = tf.convert_to_tensor(two_simplex, dtype='float32')
    eta2 = 1.- eta1
    t = tf.fill((self.num_points,), t)
    x = tf.fill((self.num_points,), x)
    y = self.dgm_model(t,x,eta1,eta2)
    y_for_graph = y.numpy()[:,0]
    return two_simplex, y_for_graph

  def single_dgm_graph_population_updated(self, t=0., x=0.):
    two_simplex = np.linspace(start = 0, stop = 1, num = self.num_points, endpoint = True)
    eta_data = np.zeros((self.num_points, d))
    eta_data[:,0] = two_simplex
    eta_data[:,1] = 1. - two_simplex
    eta_data = tf.convert_to_tensor(eta_data, dtype='float32')
    t = tf.fill((self.num_points,), t)
    x = tf.fill((self.num_points,), x)
    y = self.dgm_model(t,x,eta_data)
    y_for_graph = y.numpy()[:,0]
    return two_simplex, y_for_graph

  def single_dbmfg_graph_population(self, model_number=0, x=0.):
    two_simplex = np.linspace(start = 0, stop = 1, num = self.num_points, endpoint = True)
    data_for_graph = np.zeros((self.num_points, d+2))
    eta_data = np.zeros((len(two_simplex), d))
    eta_data[:,0] = two_simplex
    eta_data[:,1] = 1.-two_simplex
    eta_data = tf.convert_to_tensor(eta_data, dtype='float32')
    x = tf.fill((self.num_points,), x)
    y = self.dbmfg_model_list[model_number](x,eta_data)
    y_for_graph = y.numpy()[:,0]
    return two_simplex, y_for_graph

  def display_single_dgm_graph(self, t=0., x=0.):
    two_simplex, y_for_graph = self.single_dgm_graph_population(t=t, x=x)
    fig, ax = plt.subplots(figsize=(6, 4)) #, tight_layout=True)
    ax.set_ylim([0,1])
    ax.plot(two_simplex, y_for_graph)
    ax.set_xlabel(f'$\mu(x=0)$')
    ax.set_ylabel(f'$U(t={round(t,2)},x={int(x)},\eta=\mu)$')
    ax.set_title(r'')
    ax.plot(two_simplex, y_for_graph, color = 'black')
    return

  def display_single_dbmfg_graph(self, model_number=0, x=0.):
    two_simplex, y_for_graph = self.single_dbmfg_graph_population(model_number = model_number, x=x)
    fig, ax = plt.subplots(figsize=(6, 4)) #, tight_layout=True)
    ax.set_ylim([0,1])
    ax.plot(two_simplex, y_for_graph)
    ax.set_xlabel(f'$\mu(x=0)$')
    ax.set_ylabel(f'$U(t=,x={int(x)},\eta=\mu)$')
    ax.set_title(r'')
    ax.plot(two_simplex, y_for_graph, color = 'black')
    return

  def graph_errors_over_time(self, x=0., T=0.5):
    # graphing average model difference over time
    time_points = np.linspace(0, T, num = len(self.dbmfg_model_list), endpoint = True)
    average_differences = np.zeros(len(time_points))
    for k,t in enumerate(time_points):
      two_simplex, y_dgm = self.single_dgm_graph_population(t=time_points[k], x=x)
      two_simplex, y_dbmfg = self.single_dbmfg_graph_population(model_number=k, x=x)
      average_differences[k] = np.mean(y_dgm-y_dbmfg)
    fig, ax = plt.subplots(figsize=(6, 4)) #, tight_layout=True)
    ax.set_ylim([-1,1])
    ax.plot(time_points, average_differences)
    ax.set_xlabel(f'$t$')
    ax.set_ylabel(f'Avg Difference: DGME - DBME')
    ax.set_title(r'')
    return

In [ ]:
class VizSeveral:
  def __init__(self, dgm_model, dbmfg_model_array = None, num_measure_points =1_000):
    self.dgm_model = dgm_model
    self.num_points = num_measure_points
    self.dbmfg_model_list_of_lists = dbmfg_model_array
    self.viz_both_list = []
    for dbmfg_model in dbmfg_model_array:
      vb = VizBoth(dgm_model=dgm_model, dbmfg_model_list=dbmfg_model)
      self.viz_both_list.append(vb)

  def display_dgm_graph_rainbow_updated(self, x=0., T = .5):
    num_meshes = len(self.dbmfg_model_list_of_lists)
    fig, ax = plt.subplots(figsize=(6, 4)) #, tight_layout=True)
    ax.set_ylim([-0.3,0.3])
    ax.set_xlabel(f'$t$')
    ax.set_title(f'Avg Difference: DGME - DBME')

    color = cm.rainbow(np.linspace(0,1,len(self.dbmfg_model_list_of_lists)))

    avg_diff_list = []
    time_pts_list = []
    for i, dbmfg_model_list in enumerate(self.dbmfg_model_list_of_lists):
      time_points = np.linspace(0, T, num = len(dbmfg_model_list), endpoint = True)
      time_pts_list.append(time_points)
      average_differences = np.zeros(len(time_points))
      for k,t in enumerate(time_points):
        two_simplex, y_dgm = self.viz_both_list[i].single_dgm_graph_population_updated(t=time_points[k], x=x)
        two_simplex, y_dbmfg = self.viz_both_list[i].single_dbmfg_graph_population(model_number=k, x=x)
        average_differences[k] = np.mean(y_dgm-y_dbmfg)
      avg_diff_list.append(average_differences)
    partition_size = [0.1, 0.05, 0.01, 0.005, 0.001]
    for i, avg_diff in enumerate(avg_diff_list):
      ax.plot(time_pts_list[i], avg_diff, color = color[i], label = f'Partition step: {partition_size[i]}')
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    return

Below is some sample code for how to implement these classes.

Loading a saved DGME model:

In [ ]:
# loaded_dgm_model = keras.models.load_model('LOCAL PATH/Most_Models/i_dgm_max_2')
# loaded_dgm_model.compile()

Loading in DBME meshes corresponding to different partition sizes (can take time when loading many).

In [1]:
# num_models = [6, 11, 21]
# loaded_dbmfg_model_list_of_lists = []
# for i in range(3):
#   loaded_dbmfg_model_list = []
#   filepath = f'LOCAL PATH/Most_Models/mesh_models{i}'
#   for k in range(num_models[i]):
#     loaded_dbmfg_model = keras.models.load_model(filepath+'/model_tf_'+str(k))
#     loaded_dbmfg_model.compile()
#     loaded_dbmfg_model_list.append(loaded_dbmfg_model)
#   loaded_dbmfg_model_list_of_lists.append(loaded_dbmfg_model_list)

Making a VizBoth object and displaying the corresponding graphs:

In [ ]:
# viz = VizBoth(dgm_model = loaded_dgm_model, dbmfg_model_list = loaded_dbmfg_model_list)
# plt.clf()
# # viz.display_single_dbmfg_graph(model_number=11)
# # viz.display_single_dgm_graph(t=0.25)
# viz.graph_errors_over_time()

Making a VizSeveral object and displaying its comparative graph:

In [ ]:
# vizall = VizSeveral(dgm_model = loaded_dgm_model, dbmfg_model_array = loaded_dbmfg_model_list_of_lists)
# vizall.display_dgm_graph_rainbow_updated()